In [1]:
import numpy as np
import matplotlib.pyplot as plt
import data
import wifi_label_utils

In [2]:
nz_anom_metadata_filename = "data/anomwifi-encoding-0-sendsps-20000000.s16.zst.sigmf-meta"

In [3]:
wifi_data = data.Data(nz_anom_metadata_filename)

#wifi_samples = wifi_data.get_samples(n_samples=int(wifi_data.metadata["global"]["core:sample_rate"]*1))
wifi_samples = wifi_data.get_samples()
print(f"{wifi_samples.shape=}")

wifi_avg_pwr = wifi_label_utils.moving_average(wifi_samples, 256)
wifi_avg_pwr_db = 10*np.log10(wifi_avg_pwr)

# plt.figure()
# plt.plot(wifi_avg_pwr_db[:int(2*20480000e-2)])
# plt.show()

idx = wifi_label_utils.power_squelch(wifi_samples, threshold=-56, window=256)




Loaded 
 Data file: data/anomwifi-encoding-0-sendsps-20000000.s16.zst.sigmf-data 
 SigMF-Meta file: data/anomwifi-encoding-0-sendsps-20000000.s16.zst.sigmf-meta



/Users/ltindall/.pyenv/versions/3.10.4/envs/gamutrf/lib/python3.10/site-packages/sigmf/sigmffile.py:876: UserWarning: Compliant Dataset `data/anomwifi-encoding-0-sendsps-20000000.s16.zst.sigmf-data` exists but "core:dataset" is also defined; using `data/anomwifi-encoding-0-sendsps-20000000.s16.zst.sigmf-data`
  warnings.warn(f'Compliant Dataset `{compliant_data_fn}` exists but '


wifi_samples.shape=(409600000,)


In [4]:
wifi_data.sigmf_obj._metadata[wifi_data.sigmf_obj.ANNOTATION_KEY] = []
for start,stop in idx:
    start, stop = int(start), int(stop)
    metadata = {
        "core:label": "anom_wifi",
        "core:freq_lower_edge": wifi_data.metadata["captures"][0]["core:frequency"] - wifi_data.metadata["global"]["core:sample_rate"]/2,
        "core:freq_upper_edge": wifi_data.metadata["captures"][0]["core:frequency"] + wifi_data.metadata["global"]["core:sample_rate"]/2,
    }
    wifi_data.sigmf_obj.add_annotation(start, length=stop-start, metadata=metadata)


In [5]:
wifi_data.sigmf_obj.tofile(wifi_data.sigmf_meta_filename)